In [1]:
import itertools
import os
from copy import deepcopy
from typing import Any, Dict, List
import numpy as np

from fvcore.common.file_io import PathManager

from meddlr.config import get_cfg
from meddlr.evaluation.testing import find_weights
from meddlr.utils.general import find_experiment_dirs
from meddlr.utils import env

import pprint
import re

/bmrNAS/people/arjun/miniconda3/envs/meddlr_env_cpu/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Config construction utils
def configure_params(params: List[Dict], fixed=None):
    """Unroll parameters into a list of configurations."""
    configs = itertools.product(*list(params.values()))
    configs = [{k: v for k, v in zip(params.keys(), cfg)} for cfg in configs]
    if fixed is not None:
        for c in configs:
            c.update(fixed)
    return configs

def stringify_collections(value, depth=0):
    if not isinstance(value, (set, tuple, list, dict)):
        return value
    if isinstance(value, dict):
        keys = list(value.keys())
        values = [value[k] for k in keys]
        keys_str = [stringify_collections(k, depth=depth+1) for k in keys]
        values_str = [stringify_collections(v, depth=depth+1) for v in values]
        keys_str = _to_str(keys_str, keys)
        values_str = _to_str(values_str, values)
        val_dict_str = {f"{k}:{v}" for k, v in zip(keys_str, values_str)}
        value_str = "\{"
        value_str += ",".join(val_dict_str)
        value_str += "\}"
    else:
        all_vals = [stringify_collections(v, depth=depth+1) for v in value]
        value_str = ",".join(
            str(v) if not isinstance(v, str) or isinstance(ov, (set, tuple, list, dict))
            else f"'\"'\"'{v}'\"'\"'"
            for v, ov in zip(all_vals, value)
        )
        if isinstance(value, tuple):
            value_str = f"\({value_str},\)" if len(value) > 0 else f"\(\)"
        else:
            value_str = f"[{value_str}]"

    # value_str = "[" + ",".join(str(v) if not isinstance(v, str) else f"'\"'\"'{v}'\"'\"'" for v in all_vals) + "]"
    return value_str

def _to_str(all_vals, value):
    return [
        str(v) if not isinstance(v, str) or isinstance(ov, (set, tuple, list, dict))
        else f"'\"'\"'{v}'\"'\"'" for v, ov in zip(all_vals, value)
    ]

def stringify(cfg: Dict[str, Any]):
    cfg = {k: f"'{v}'" if isinstance(v, str) else v for k, v in cfg.items()}
    cfg = {k: stringify_collections(v) for k, v in cfg.items()}
    return " ".join(f"{k} \"{v}\"" for k, v in cfg.items())

def get_next_version(path):
    path = PathManager.get_local_path(path)
    if not os.path.isdir(path):
        max_version = 0
    else:
        versions = [int(x.split("_")[1])  for x in os.listdir(path) if x.startswith("version")]
        max_version = max(versions) if versions else 0
    return max_version + 1

def format_version_str(version):
    return f"version_{version:03d}"

def format_path(path):
    path = PathManager.get_local_path(path)
    next_version = get_next_version(path)
    return os.path.join(path, format_version_str(next_version))

print(stringify_collections([["data", "nothing"]]))
print(stringify_collections({"data": "nothing"}))
print(stringify_collections(("hi",)))

[['"'"'data'"'"','"'"'nothing'"'"']]
\{'"'"'data'"'"':'"'"'nothing'"'"'\}
\('"'"'hi'"'"',\)


In [3]:
def run_experiment(cfg_filename):
    base_cfg = get_cfg()
    base_cfg.merge_from_file(cfg_filename)

    for exp_type, exp_vars in EXP_TYPES.items():
        base_out_path = EXP_OUT_DIR.format(exp_type=exp_type)
        if DEBUG:
            base_out_path += "-debug"
        descriptions = exp_vars.get("desc", [None] * len(exp_vars["values"]))
        filter_by = list(base_cfg.DATALOADER.FILTER.BY) if not exp_vars.get("del_filter", False) else []

        params = {
            "DATALOADER.FILTER.BY": [filter_by + [(exp_vars["filter_by"], v)] if v is not None else list(filter_by) for v in exp_vars["values"]]
        }

        fixed_params = {
            # Description information
            "DESCRIPTION.BRIEF": base_cfg.DESCRIPTION.BRIEF.format(exp_type=exp_type),
            "DESCRIPTION.TAGS": list(set(list(base_cfg.DESCRIPTION.TAGS) + [exp_type])),
        }
        fixed_params.update(exp_vars.get("fixed_params", {}))

        all_configs = configure_params(params, fixed_params)
        for idx, (cfg, desc) in enumerate(zip(all_configs, descriptions)):
            version_str = format_version_str(get_next_version(base_out_path))
            out_path = PathManager.get_local_path(os.path.join(base_out_path, version_str))

            # CAUTION: LOOK HERE BEFORE RUNNING
            cfg["OUTPUT_DIR"] = out_path
            exp_suffix = f"{desc}/{version_str}" if desc else f"{version_str}"
            cfg["DESCRIPTION.EXP_NAME"] = EXP_NAME.format(exp_type=exp_type, suffix=exp_suffix)
            # if desc:
            #     cfg["DESCRIPTION.TAGS"] = cfg.get("DESCRIPTION.TAGS", base_cfg.DESCRIPTION.TAGS) + [desc]
        
            cfg_args = stringify(cfg)

            cfg_list = [item for x in [(k, v) for k, v in cfg.items()] for item in x]
            cfg_obj = base_cfg.clone()
            cfg_obj.merge_from_list(cfg_list)

            cfg_str = "\n\t".join(f"{k}: {v}" for k, v in cfg.items())
            print(f"({idx+1}/{len(all_configs)}) {out_path}:\n\t{cfg_str}")
            print("")
            #print(f"\t{cfg_args}")

            if SUBMIT_JOBS:
                num_workers = max(cfg_obj.DATALOADER.NUM_WORKERS, 2)
                debug = "--debug" if DEBUG else ""
                os.makedirs(out_path, exist_ok=True)
                !sbatch -c $num_workers --mem-per-cpu=8gb train.sh \
                    SSRECON_MPROFILE=$PROFILE_MEMORY python train_net.py --config-file $cfg_filename --reproducible $debug $cfg_args
            
            print("=="*40)
            print("")


## VORTEX-RM Experiments

In [7]:
cfg_dir = "/bmrNAS/people/arjun/code/projects/meddlr-configs-deepro/mridata-3dfse-knee/unrolled/ssdu"
cfgs = []
for root, dirs, files in os.walk(cfg_dir):
    for file in files:
        if file.endswith('.yaml'):
            cfgs.append(os.path.join(root, file))

for cfg_file in cfgs:
    !sbatch -c 8 --mem-per-cpu=5gb --nodelist=amalfi /bmrNAS/people/arjun/code/meddlr/tools/train.sh \
        python /bmrNAS/people/arjun/code/meddlr/tools/train_net.py \
        --config-file $cfg_file \
        --repro --auto-version

Submitted batch job 112219
Submitted batch job 112220
Submitted batch job 112221
Submitted batch job 112222
